# IDS 561 Homework 2
Isaac Salvador<br>UIN: 669845132

## Dataset
We first instantiate a `pyspark.sql` session to work in a dataframe environment and load the `Amazon_Responded_Oct05.csv` file. The argument `inferSchema=True` allows spark to automatically infer the data types and columns of the dataframe.

In [1]:
from pyspark.sql import SparkSession

# start spark session
sc = SparkSession.builder.appName("Homework2").getOrCreate()

file_path = "data/Amazon_Responded_Oct05.csv"
amazon_df = sc.read.csv(file_path, header=True, inferSchema=True).na.drop()

23/11/06 21:38:06 WARN Utils: Your hostname, Isaacs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.165 instead (on interface en0)
23/11/06 21:38:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/06 21:38:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Next, we select the relevant columns `"tweet_created_at"`, `"user_screen_name"`, and `"user_id_str"` for further analysis. The field  `"tweet_created_at"` is subseqeuntly converted from `string` data type to `TimestampType` data type.

In [2]:
# choose relevant columns
relevant_columns = ["tweet_created_at", "user_screen_name", "user_id_str"]

# select relevant columns
amazon_analysis_df = amazon_df.select(relevant_columns)

# convert "tweet_created_at" string column to date column
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType
from datetime import datetime

# create parsing function to convert from string to timestamp
def parse_tweet_created_at(date_str):
    try:
        return datetime.strptime(date_str, '%a %b %d %H:%M:%S %z %Y')
    except ValueError:
        return None

# create a spark user defined function (UDF)
parse_tweet_created_at_udf = udf(parse_tweet_created_at, TimestampType())

# apply UDF to "tweet_created_at function"
amazon_analysis_df = amazon_analysis_df.withColumn(
    "tweet_created_at",
    parse_tweet_created_at_udf(
        amazon_analysis_df['tweet_created_at']
    )
)

# remove null values
amazon_analysis_filtered_df = amazon_analysis_df.filter(
    amazon_analysis_df['tweet_created_at'].isNotNull()
)

## Tasks
### _Step 1_
The first step is to `groupby` distinct users and there corresponding user id's and obtain the number of unique days the users have tweeted. We then `filter` to obtain only the users that have tweeted at least 5 times. Per this methodology there were 262 users that tweeted at least 5 times. These results are then exported to csv format.

In [3]:
from pyspark.sql.functions import col, count

# group users and sum the number of times they tweeted
daily_active_users = amazon_analysis_filtered_df\
    .distinct()\
    .groupby(["user_screen_name", "user_id_str"])\
    .agg(count("tweet_created_at").alias("number_of_days"))

# filter for users that have tweeted at least 5 times
daily_active_users = daily_active_users.filter(col("number_of_days") >= 5)

# select relevant columns
daily_active_users = daily_active_users.select(
    ["user_screen_name", "user_id_str"]
)

print(f"Number of Daily Active Users: {daily_active_users.count()}")

# export to csv
#daily_active_users.write.csv("data/HW3_output/daily_active_users.csv",header=True) 

23/11/06 21:38:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Number of Daily Active Users: 262


### _Step 2_
To conduct A/B testing on active users, we first load the `experiment.txt` file and convert it to a spark dataframe object. We additionally specify a `schema` such that the column contains `StringType` data types and has the column name `user_id_str`.

In [4]:
from pyspark.sql.types import StructType, StructField, StringType

# create custom schema
schema = StructType([
    StructField(
        "user_id_str",
        StringType(),
        True
    )
])

# load experiment.txt
file_path = "data/experiment.txt"
experiment_df = sc.read.csv(file_path, header = False, schema=schema)

We next perform a left join on `experiment_from_txt` with `daily_active_users` to generate a new column `"whether_active"` using the pyspark function `when`.

In [5]:
from pyspark.sql.functions import when

# perform left join on experiment_from_text with daily_active_users
experiment_join_df = experiment_df.join(daily_active_users, 'user_id_str', how="left")


# generate new column "whether_active" based on null values after left join
experiment_join_null_check_df = experiment_join_df.withColumn(
    "whether_active",
    when(col("user_screen_name").isNull(), "No").otherwise('yes')
)

# select only "user_id_str" and "whether_active"
experiment = experiment_join_null_check_df.drop("user_screen_name")

# export to csv
#experiment.write.csv("data/HW3_output/experiment.csv",header=True)

We can then calculate the percentage of active users using filter arguments. Out of the 5,000 users in the `experiment.txt` file, 0.22% were active.

In [6]:
# calculate the percent of active users
percent_active = \
    experiment.filter(col("whether_active")=="yes").count()/\
    experiment.count()

formatted_percentage = "{:.2f}%".format(percent_active*100)
print("Percent Active: "+formatted_percentage)

Percent Active: 0.22%


### _Step 3_
To obtain the desired output we first download the `final_experiment.txt` file and convert to a spark dataframe object. We then drop the `"whether_active"` and `user_screen_name` fields which will be repopulated following subsequent join operations.

In [7]:
# load final_experiment.csv file
file_path = "data/final_experiment.csv"
final_experiment_df = sc.read.csv(file_path, header=True)

# drop fields
final_experiment_df = final_experiment_df.drop("user_screen_name").drop("whether_active")

We then use join `final_experiment_df` and the `experiment` dataframe from _Step 2_ to populate the empty row `"whether_active"`. We perform another left join to obtain `"user_screen_name"` from the original `amazon_df`.

In [8]:
# join on "user_id_str"
joined_df = final_experiment_df.join(
    experiment, "user_id_str", "left"
)

joined_df = joined_df.join(
    amazon_analysis_df.select(["user_id_str", "user_screen_name"]), "user_id_str", "left"
)

Finally, we replace all null values from the resultant joins with the string `"Not found"`.

In [9]:
final_user_experiment = joined_df.na.fill("Not found")

# export to csv
#final_user_experiment.write.csv("data/HW3_output/final_user_experiment.csv", header=True)

# end spark session
sc.stop()